# Lab 11: Advanced Ransomware Detection & Response

Build an AI-powered system to detect, analyze, and respond to modern ransomware attacks.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab11_ransomware_detection.ipynb)

## Learning Objectives
- Multi-stage ransomware behavioral detection
- Entropy-based and pattern-based encryption detection
- Ransomware family classification (LockBit, BlackCat/ALPHV, Conti, Royal, Play, Akira)
- Shadow copy & recovery inhibition detection
- Ransom note analysis and IOC extraction
- Decryptor availability checking
- Automated response playbooks with MITRE ATT&CK mapping

## Modern Ransomware Landscape

Modern ransomware operates in sophisticated multi-stage attacks:
1. **Initial Access** - Phishing, RDP brute force, exploits
2. **Reconnaissance** - Network enumeration, file discovery
3. **Lateral Movement** - Credential theft, PSExec, WMI
4. **Data Exfiltration** - Double extortion preparation
5. **Encryption** - File encryption with recovery inhibition
6. **Extortion** - Ransom demands, data leak threats

In [ ]:
import math
import re
import json
import hashlib
from collections import Counter
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta
from enum import Enum
import random

# For visualization
import pandas as pd
import numpy as np

class RansomwareFamily(Enum):
    """Known ransomware families with characteristics."""
    LOCKBIT = "lockbit"
    BLACKCAT = "blackcat"  # Also known as ALPHV
    CONTI = "conti"
    ROYAL = "royal"
    PLAY = "play"
    AKIRA = "akira"
    BLACKBASTA = "blackbasta"
    CLOP = "clop"
    RHYSIDA = "rhysida"
    UNKNOWN = "unknown"

# Ransomware family characteristics database
RANSOMWARE_FAMILIES = {
    RansomwareFamily.LOCKBIT: {
        "extensions": [".lockbit", ".abcd", ".LockBit"],
        "note_patterns": ["lockbit", "restore-my-files", "LOCKBIT 3.0"],
        "encryption": "AES-256 + RSA-2048",
        "double_extortion": True,
        "decryptor_available": False,
        "avg_ransom_usd": 250000,
        "ttps": ["T1486", "T1490", "T1027", "T1055"]
    },
    RansomwareFamily.BLACKCAT: {
        "extensions": [".alphv", ".ALPHV"],
        "note_patterns": ["alphv", "blackcat", "RECOVER-FILES"],
        "encryption": "ChaCha20 + RSA",
        "double_extortion": True,
        "decryptor_available": False,
        "avg_ransom_usd": 400000,
        "ttps": ["T1486", "T1490", "T1070", "T1562"]
    },
    RansomwareFamily.CONTI: {
        "extensions": [".CONTI", ".conti"],
        "note_patterns": ["conti", "CONTI_README"],
        "encryption": "ChaCha20 + RSA-4096",
        "double_extortion": True,
        "decryptor_available": True,  # Leaked after group dissolution
        "avg_ransom_usd": 150000,
        "ttps": ["T1486", "T1490", "T1082", "T1083"]
    },
    RansomwareFamily.ROYAL: {
        "extensions": [".royal"],
        "note_patterns": ["royal", "README.TXT"],
        "encryption": "AES-256 + RSA",
        "double_extortion": True,
        "decryptor_available": False,
        "avg_ransom_usd": 350000,
        "ttps": ["T1486", "T1490", "T1059"]
    },
    RansomwareFamily.PLAY: {
        "extensions": [".play", ".PLAY"],
        "note_patterns": ["play", "ReadMe.txt"],
        "encryption": "AES + RSA",
        "double_extortion": True,
        "decryptor_available": False,
        "avg_ransom_usd": 300000,
        "ttps": ["T1486", "T1490", "T1218"]
    },
    RansomwareFamily.AKIRA: {
        "extensions": [".akira"],
        "note_patterns": ["akira", "akira_readme"],
        "encryption": "ChaCha20 + RSA",
        "double_extortion": True,
        "decryptor_available": True,  # Avast released decryptor
        "avg_ransom_usd": 200000,
        "ttps": ["T1486", "T1490", "T1021"]
    }
}

print(f"Loaded {len(RANSOMWARE_FAMILIES)} ransomware family profiles")

## 1. Entropy-Based Detection

In [ ]:
def calculate_entropy(data: bytes) -> float:
    """Calculate Shannon entropy of data (0-8 scale for bytes)."""
    if not data:
        return 0.0
    
    counter = Counter(data)
    length = len(data)
    entropy = 0.0
    
    for count in counter.values():
        probability = count / length
        if probability > 0:
            entropy -= probability * math.log2(probability)
    
    return entropy

def analyze_file_header(data: bytes) -> Dict:
    """Analyze file header for encryption indicators."""
    if len(data) < 16:
        return {"valid_header": False, "file_type": "unknown"}
    
    # Common file signatures
    signatures = {
        b'\x50\x4B\x03\x04': "ZIP/DOCX/XLSX",
        b'\x25\x50\x44\x46': "PDF",
        b'\x89\x50\x4E\x47': "PNG",
        b'\xFF\xD8\xFF': "JPEG",
        b'\x4D\x5A': "PE Executable",
        b'\x7F\x45\x4C\x46': "ELF",
        b'\xD0\xCF\x11\xE0': "MS Office (old)",
    }
    
    for sig, file_type in signatures.items():
        if data[:len(sig)] == sig:
            return {"valid_header": True, "file_type": file_type}
    
    return {"valid_header": False, "file_type": "unknown/encrypted"}

# Test entropy with realistic samples
test_cases = [
    ("Normal text file", b"Hello, this is a normal text file with regular content." * 10),
    ("Structured data (JSON)", b'{"name": "John", "age": 30, "city": "NYC"}' * 10),
    ("Compressed (low entropy)", bytes([0x1F, 0x8B] + [0x08] * 100)),  # gzip-like header
    ("Partially encrypted", bytes(range(128)) + b"ENCRYPTED_MARKER" + bytes(range(128, 256))),
    ("Fully encrypted (random)", bytes([random.randint(0, 255) for _ in range(500)])),
    ("Ransomware encrypted", bytes([random.randint(0, 255) for _ in range(450)]) + b".lockbit"),
]

print("Entropy Analysis for Various File Types:")
print("=" * 70)
print(f"{'Sample Type':<30} {'Entropy':<10} {'Assessment':<30}")
print("-" * 70)
for name, data in test_cases:
    entropy = calculate_entropy(data)
    header = analyze_file_header(data)
    
    if entropy > 7.5:
        assessment = "HIGH ENTROPY - Likely encrypted"
    elif entropy > 6.5:
        assessment = "ELEVATED - Possibly compressed/encrypted"
    elif entropy > 4.0:
        assessment = "MODERATE - Normal binary/text"
    else:
        assessment = "LOW - Repetitive/structured data"
    
    print(f"{name:<30} {entropy:<10.3f} {assessment:<30}")

## 2. Multi-Stage Attack Simulation

Simulate realistic ransomware attack chains with multiple stages.

In [ ]:
@dataclass
class FileEvent:
    """Enhanced file system event with attack context."""
    timestamp: str
    process: str
    process_pid: int
    operation: str
    file_path: str
    original_extension: str
    new_extension: str
    entropy_before: float
    entropy_after: float
    file_size: int
    parent_process: str
    command_line: str
    user: str

@dataclass
class AttackStage:
    """Ransomware attack stage."""
    stage: str
    description: str
    mitre_technique: str
    events: List[FileEvent]

def generate_realistic_attack_simulation(family: RansomwareFamily) -> List[AttackStage]:
    """Generate a realistic multi-stage ransomware attack simulation."""
    
    family_info = RANSOMWARE_FAMILIES.get(family, {})
    extension = family_info.get("extensions", [".encrypted"])[0]
    
    # Common target files in enterprise environments
    target_files = [
        ("C:\\Users\\jsmith\\Documents\\Q4_Financial_Report.xlsx", ".xlsx", 156000),
        ("C:\\Users\\jsmith\\Documents\\Employee_Database.accdb", ".accdb", 2340000),
        ("C:\\Users\\jsmith\\Desktop\\Contracts\\Vendor_Agreement.docx", ".docx", 89000),
        ("C:\\Users\\jsmith\\Desktop\\Contracts\\NDA_Template.pdf", ".pdf", 234000),
        ("C:\\Shares\\Finance\\Budget_2024.xlsx", ".xlsx", 445000),
        ("C:\\Shares\\Finance\\Payroll_Q3.csv", ".csv", 123000),
        ("C:\\Shares\\HR\\Personnel_Records.xlsx", ".xlsx", 567000),
        ("C:\\Shares\\Engineering\\Source_Code_Backup.zip", ".zip", 89000000),
        ("C:\\Shares\\Legal\\Patent_Applications.pdf", ".pdf", 12300000),
        ("C:\\Shares\\Marketing\\Campaign_Assets.psd", ".psd", 45600000),
        ("D:\\Backups\\SQL_Backup_20240115.bak", ".bak", 234000000),
        ("D:\\Backups\\Exchange_Backup.edb", ".edb", 567000000),
        ("E:\\VMs\\DC01.vmdk", ".vmdk", 89000000000),
        ("C:\\Users\\admin\\Documents\\IT_Credentials.kdbx", ".kdbx", 45000),
        ("C:\\ProgramData\\Application\\config.db", ".db", 12000),
    ]
    
    stages = []
    base_time = datetime(2024, 1, 15, 9, 0, 0)
    
    # Stage 1: Initial Access & Reconnaissance
    stage1_events = [
        FileEvent(
            timestamp=(base_time + timedelta(minutes=0)).strftime("%H:%M:%S"),
            process="outlook.exe", process_pid=4521,
            operation="CREATE",
            file_path="C:\\Users\\jsmith\\AppData\\Local\\Temp\\Invoice_Jan2024.docm",
            original_extension="", new_extension=".docm",
            entropy_before=0, entropy_after=5.2,
            file_size=156000, parent_process="explorer.exe",
            command_line="outlook.exe", user="CORP\\jsmith"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=1)).strftime("%H:%M:%S"),
            process="WINWORD.EXE", process_pid=5634,
            operation="EXECUTE",
            file_path="C:\\Users\\jsmith\\AppData\\Local\\Temp\\Invoice_Jan2024.docm",
            original_extension=".docm", new_extension=".docm",
            entropy_before=5.2, entropy_after=5.2,
            file_size=156000, parent_process="outlook.exe",
            command_line="\"C:\\Program Files\\Microsoft Office\\WINWORD.EXE\" /n Invoice_Jan2024.docm",
            user="CORP\\jsmith"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=2)).strftime("%H:%M:%S"),
            process="powershell.exe", process_pid=6745,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.1, entropy_after=6.1,
            file_size=450000, parent_process="WINWORD.EXE",
            command_line="powershell.exe -ep bypass -enc SQBFAFgAIAAoAE4AZQB3AC...",
            user="CORP\\jsmith"
        ),
    ]
    stages.append(AttackStage("Initial Access", "Macro-enabled document execution", "T1566.001", stage1_events))
    
    # Stage 2: Discovery & Credential Access
    stage2_events = [
        FileEvent(
            timestamp=(base_time + timedelta(minutes=5)).strftime("%H:%M:%S"),
            process="net.exe", process_pid=7823,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\net.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.2, entropy_after=6.2,
            file_size=45000, parent_process="powershell.exe",
            command_line="net view /domain", user="CORP\\jsmith"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=6)).strftime("%H:%M:%S"),
            process="nltest.exe", process_pid=7834,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\nltest.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.1, entropy_after=6.1,
            file_size=67000, parent_process="powershell.exe",
            command_line="nltest /dclist:corp.local", user="CORP\\jsmith"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=8)).strftime("%H:%M:%S"),
            process="mimikatz.exe", process_pid=7845,
            operation="EXECUTE",
            file_path="C:\\Users\\jsmith\\AppData\\Local\\Temp\\m.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=7.2, entropy_after=7.2,
            file_size=1200000, parent_process="powershell.exe",
            command_line="m.exe sekurlsa::logonpasswords", user="CORP\\jsmith"
        ),
    ]
    stages.append(AttackStage("Discovery", "Network and credential enumeration", "T1087", stage2_events))
    
    # Stage 3: Defense Evasion & Persistence
    stage3_events = [
        FileEvent(
            timestamp=(base_time + timedelta(minutes=12)).strftime("%H:%M:%S"),
            process="reg.exe", process_pid=8912,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\reg.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.0, entropy_after=6.0,
            file_size=78000, parent_process="powershell.exe",
            command_line="reg add HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run /v Update /d C:\\ProgramData\\svc.exe",
            user="CORP\\admin"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=13)).strftime("%H:%M:%S"),
            process="powershell.exe", process_pid=8934,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.1, entropy_after=6.1,
            file_size=450000, parent_process="cmd.exe",
            command_line="powershell.exe Set-MpPreference -DisableRealtimeMonitoring $true",
            user="CORP\\admin"
        ),
    ]
    stages.append(AttackStage("Defense Evasion", "Security tool disabling", "T1562.001", stage3_events))
    
    # Stage 4: Recovery Inhibition (Critical Ransomware Indicator)
    stage4_events = [
        FileEvent(
            timestamp=(base_time + timedelta(minutes=20)).strftime("%H:%M:%S"),
            process="vssadmin.exe", process_pid=9123,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\vssadmin.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.0, entropy_after=6.0,
            file_size=234000, parent_process="cmd.exe",
            command_line="vssadmin.exe delete shadows /all /quiet",
            user="CORP\\admin"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=21)).strftime("%H:%M:%S"),
            process="wmic.exe", process_pid=9134,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\wbem\\wmic.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.2, entropy_after=6.2,
            file_size=456000, parent_process="cmd.exe",
            command_line="wmic shadowcopy delete",
            user="CORP\\admin"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=22)).strftime("%H:%M:%S"),
            process="bcdedit.exe", process_pid=9145,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\bcdedit.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=5.8, entropy_after=5.8,
            file_size=123000, parent_process="cmd.exe",
            command_line="bcdedit /set {default} recoveryenabled no",
            user="CORP\\admin"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=23)).strftime("%H:%M:%S"),
            process="wbadmin.exe", process_pid=9156,
            operation="EXECUTE",
            file_path="C:\\Windows\\System32\\wbadmin.exe",
            original_extension=".exe", new_extension=".exe",
            entropy_before=6.0, entropy_after=6.0,
            file_size=234000, parent_process="cmd.exe",
            command_line="wbadmin delete catalog -quiet",
            user="CORP\\admin"
        ),
    ]
    stages.append(AttackStage("Recovery Inhibition", "Backup and shadow copy destruction", "T1490", stage4_events))
    
    # Stage 5: Data Encryption
    encryption_events = []
    for i, (filepath, orig_ext, size) in enumerate(target_files):
        encryption_events.append(FileEvent(
            timestamp=(base_time + timedelta(minutes=25, seconds=i*2)).strftime("%H:%M:%S.%f")[:-3],
            process=f"{family.value}.exe", process_pid=9500,
            operation="ENCRYPT",
            file_path=filepath + extension,
            original_extension=orig_ext, new_extension=extension,
            entropy_before=random.uniform(4.0, 6.5),
            entropy_after=random.uniform(7.85, 7.99),
            file_size=size, parent_process="svchost.exe",
            command_line=f"C:\\ProgramData\\{family.value}.exe --encrypt --path C:\\ --ext {extension}",
            user="SYSTEM"
        ))
    stages.append(AttackStage("Encryption", "File encryption with high entropy", "T1486", encryption_events))
    
    # Stage 6: Ransom Note Drop
    stage6_events = [
        FileEvent(
            timestamp=(base_time + timedelta(minutes=35)).strftime("%H:%M:%S"),
            process=f"{family.value}.exe", process_pid=9500,
            operation="CREATE",
            file_path="C:\\Users\\jsmith\\Desktop\\README_RESTORE_FILES.txt",
            original_extension="", new_extension=".txt",
            entropy_before=0, entropy_after=4.2,
            file_size=2500, parent_process="svchost.exe",
            command_line=f"C:\\ProgramData\\{family.value}.exe --note",
            user="SYSTEM"
        ),
        FileEvent(
            timestamp=(base_time + timedelta(minutes=35, seconds=1)).strftime("%H:%M:%S"),
            process=f"{family.value}.exe", process_pid=9500,
            operation="CREATE",
            file_path="C:\\Users\\jsmith\\Documents\\README_RESTORE_FILES.txt",
            original_extension="", new_extension=".txt",
            entropy_before=0, entropy_after=4.2,
            file_size=2500, parent_process="svchost.exe",
            command_line=f"C:\\ProgramData\\{family.value}.exe --note",
            user="SYSTEM"
        ),
    ]
    stages.append(AttackStage("Extortion", "Ransom note deployment", "T1491", stage6_events))
    
    return stages

# Generate attack simulation for LockBit
attack_simulation = generate_realistic_attack_simulation(RansomwareFamily.LOCKBIT)

print("Multi-Stage Ransomware Attack Simulation (LockBit)")
print("=" * 80)
total_events = 0
for stage in attack_simulation:
    print(f"\n[{stage.mitre_technique}] {stage.stage}: {stage.description}")
    print(f"  Events: {len(stage.events)}")
    for event in stage.events[:3]:  # Show first 3 events per stage
        print(f"    {event.timestamp} | {event.process} | {event.operation} | {event.file_path[-50:]}")
    if len(stage.events) > 3:
        print(f"    ... and {len(stage.events) - 3} more events")
    total_events += len(stage.events)

print(f"\nTotal events in simulation: {total_events}")

## 3. Advanced Ransomware Detection Engine

Multi-factor detection combining behavioral, entropy, and pattern analysis.

In [ ]:
# Comprehensive ransomware extension database
RANSOMWARE_EXTENSIONS = {
    # LockBit variants
    '.lockbit', '.abcd', '.LockBit',
    # BlackCat/ALPHV
    '.alphv', '.ALPHV', 
    # Other major families
    '.conti', '.CONTI', '.royal', '.play', '.PLAY', '.akira',
    '.blackbasta', '.basta', '.clop', '.cl0p', '.rhysida',
    # Historical/common
    '.locked', '.encrypted', '.crypto', '.crypt', '.enc',
    '.locky', '.cerber', '.zepto', '.odin', '.thor',
    '.aesir', '.zzzzz', '.crypted', '.crinf', '.r5a',
    '.WNCRY', '.wcry', '.wncrypt',  # WannaCry
    '.petya', '.notpetya',  # Petya variants
    '.ryuk', '.RYK',  # Ryuk
    '.maze', '.egregor',  # Maze/Egregor
    '.revil', '.sodinokibi',  # REvil
    '.hive', '.key.hive',  # Hive
    '.cuba',  # Cuba
}

# Recovery inhibition commands to detect
RECOVERY_INHIBITION_PATTERNS = [
    # Shadow copy deletion
    r'vssadmin.*delete.*shadows',
    r'wmic.*shadowcopy.*delete',
    r'vssadmin.*resize.*shadowstorage.*\/for=\w:.*\/on=\w:.*\/maxsize=',
    # Windows recovery disabling
    r'bcdedit.*\/set.*recoveryenabled.*no',
    r'bcdedit.*\/set.*bootstatuspolicy.*ignoreallfailures',
    # Backup catalog deletion
    r'wbadmin.*delete.*catalog',
    r'wbadmin.*delete.*systemstatebackup',
    # Disable Windows Error Recovery
    r'bcdedit.*\/set.*safeboot',
    # Delete backup files
    r'del.*\/s.*\/q.*\.bak',
    r'del.*\/s.*\/q.*\.backup',
]

# MITRE ATT&CK technique mapping
MITRE_TECHNIQUES = {
    "T1486": {"name": "Data Encrypted for Impact", "tactic": "Impact"},
    "T1490": {"name": "Inhibit System Recovery", "tactic": "Impact"},
    "T1489": {"name": "Service Stop", "tactic": "Impact"},
    "T1491": {"name": "Defacement", "tactic": "Impact"},
    "T1027": {"name": "Obfuscated Files or Information", "tactic": "Defense Evasion"},
    "T1055": {"name": "Process Injection", "tactic": "Defense Evasion"},
    "T1562": {"name": "Impair Defenses", "tactic": "Defense Evasion"},
    "T1070": {"name": "Indicator Removal", "tactic": "Defense Evasion"},
    "T1566": {"name": "Phishing", "tactic": "Initial Access"},
    "T1087": {"name": "Account Discovery", "tactic": "Discovery"},
    "T1083": {"name": "File and Directory Discovery", "tactic": "Discovery"},
}

class AdvancedRansomwareDetector:
    """Advanced multi-factor ransomware detection engine."""
    
    ENTROPY_THRESHOLD = 7.5
    HIGH_ENTROPY_WRITE_THRESHOLD = 5  # Number of high-entropy writes to trigger
    ENCRYPTION_RATE_THRESHOLD = 10  # Files per minute
    
    def __init__(self):
        self.detection_weights = {
            "high_entropy_writes": 0.25,
            "ransomware_extensions": 0.20,
            "recovery_inhibition": 0.25,
            "ransom_note_creation": 0.15,
            "rapid_file_modification": 0.10,
            "suspicious_process_tree": 0.05,
        }
    
    def analyze_attack(self, stages: List[AttackStage]) -> Dict:
        """Analyze complete attack chain for ransomware indicators."""
        all_events = []
        for stage in stages:
            all_events.extend(stage.events)
        
        result = {
            "is_ransomware": False,
            "confidence": 0.0,
            "risk_level": "LOW",
            "detected_family": None,
            "indicators": [],
            "mitre_techniques": [],
            "timeline": [],
            "recommendations": [],
            "decryptor_available": False
        }
        
        scores = {}
        
        # 1. High entropy writes detection
        high_entropy_writes = [
            e for e in all_events 
            if e.operation in ["WRITE", "ENCRYPT"] and e.entropy_after > self.ENTROPY_THRESHOLD
        ]
        if len(high_entropy_writes) >= self.HIGH_ENTROPY_WRITE_THRESHOLD:
            scores["high_entropy_writes"] = min(1.0, len(high_entropy_writes) / 20)
            result["indicators"].append({
                "type": "HIGH_ENTROPY_ENCRYPTION",
                "count": len(high_entropy_writes),
                "avg_entropy": sum(e.entropy_after for e in high_entropy_writes) / len(high_entropy_writes),
                "severity": "CRITICAL"
            })
            result["mitre_techniques"].append("T1486")
        
        # 2. Ransomware extension detection
        ransom_ext_events = [
            e for e in all_events 
            if e.new_extension.lower() in RANSOMWARE_EXTENSIONS
        ]
        if ransom_ext_events:
            scores["ransomware_extensions"] = min(1.0, len(ransom_ext_events) / 10)
            detected_ext = ransom_ext_events[0].new_extension
            result["indicators"].append({
                "type": "KNOWN_RANSOMWARE_EXTENSION",
                "extension": detected_ext,
                "count": len(ransom_ext_events),
                "severity": "CRITICAL"
            })
            
            # Try to identify family from extension
            for family, info in RANSOMWARE_FAMILIES.items():
                if detected_ext in info.get("extensions", []):
                    result["detected_family"] = family.value
                    result["decryptor_available"] = info.get("decryptor_available", False)
                    break
        
        # 3. Recovery inhibition detection
        recovery_inhibition = []
        for event in all_events:
            cmd = event.command_line.lower()
            for pattern in RECOVERY_INHIBITION_PATTERNS:
                if re.search(pattern, cmd, re.IGNORECASE):
                    recovery_inhibition.append(event)
                    break
        
        if recovery_inhibition:
            scores["recovery_inhibition"] = min(1.0, len(recovery_inhibition) / 4)
            result["indicators"].append({
                "type": "RECOVERY_INHIBITION",
                "commands": [e.command_line for e in recovery_inhibition[:5]],
                "severity": "CRITICAL"
            })
            result["mitre_techniques"].append("T1490")
        
        # 4. Ransom note creation detection
        note_patterns = ["readme", "restore", "decrypt", "recover", "how_to", "ransom"]
        ransom_notes = [
            e for e in all_events
            if e.operation == "CREATE" and 
            any(p in e.file_path.lower() for p in note_patterns) and
            e.new_extension == ".txt"
        ]
        if ransom_notes:
            scores["ransom_note_creation"] = 1.0
            result["indicators"].append({
                "type": "RANSOM_NOTE_DETECTED",
                "paths": [e.file_path for e in ransom_notes],
                "severity": "HIGH"
            })
            result["mitre_techniques"].append("T1491")
        
        # 5. Rapid file modification (encryption speed)
        encrypt_events = [e for e in all_events if e.operation in ["WRITE", "ENCRYPT"]]
        if len(encrypt_events) > self.ENCRYPTION_RATE_THRESHOLD:
            scores["rapid_file_modification"] = min(1.0, len(encrypt_events) / 50)
            result["indicators"].append({
                "type": "RAPID_FILE_ENCRYPTION",
                "files_modified": len(encrypt_events),
                "severity": "HIGH"
            })
        
        # 6. Suspicious process tree
        suspicious_parents = ["WINWORD.EXE", "EXCEL.EXE", "POWERPNT.EXE", "outlook.exe"]
        suspicious_children = ["powershell.exe", "cmd.exe", "wscript.exe", "cscript.exe"]
        suspicious_chains = [
            e for e in all_events
            if e.parent_process.lower() in [p.lower() for p in suspicious_parents] and
            e.process.lower() in [c.lower() for c in suspicious_children]
        ]
        if suspicious_chains:
            scores["suspicious_process_tree"] = min(1.0, len(suspicious_chains) / 3)
            result["indicators"].append({
                "type": "SUSPICIOUS_PROCESS_CHAIN",
                "chains": [f"{e.parent_process} -> {e.process}" for e in suspicious_chains[:5]],
                "severity": "MEDIUM"
            })
            result["mitre_techniques"].append("T1566")
        
        # Calculate weighted confidence score
        total_score = sum(
            scores.get(indicator, 0) * weight 
            for indicator, weight in self.detection_weights.items()
        )
        result["confidence"] = min(1.0, total_score)
        
        # Determine risk level
        if result["confidence"] >= 0.8:
            result["risk_level"] = "CRITICAL"
            result["is_ransomware"] = True
        elif result["confidence"] >= 0.6:
            result["risk_level"] = "HIGH"
            result["is_ransomware"] = True
        elif result["confidence"] >= 0.4:
            result["risk_level"] = "MEDIUM"
        else:
            result["risk_level"] = "LOW"
        
        # Build attack timeline
        for stage in stages:
            if stage.events:
                result["timeline"].append({
                    "stage": stage.stage,
                    "technique": stage.mitre_technique,
                    "start_time": stage.events[0].timestamp,
                    "event_count": len(stage.events)
                })
        
        # Generate recommendations
        result["recommendations"] = self._generate_recommendations(result)
        
        # Deduplicate MITRE techniques
        result["mitre_techniques"] = list(set(result["mitre_techniques"]))
        
        return result
    
    def _generate_recommendations(self, result: Dict) -> List[Dict]:
        """Generate response recommendations based on detection."""
        recommendations = []
        
        if result["risk_level"] in ["CRITICAL", "HIGH"]:
            recommendations.extend([
                {"priority": 1, "action": "ISOLATE", "description": "Immediately isolate affected hosts from network"},
                {"priority": 2, "action": "ALERT", "description": "Escalate to incident response team"},
                {"priority": 3, "action": "PRESERVE", "description": "Capture memory dump and disk image for forensics"},
            ])
        
        if result.get("decryptor_available"):
            recommendations.append({
                "priority": 4, 
                "action": "DECRYPT", 
                "description": f"Decryptor may be available for {result.get('detected_family', 'this family')} - check NoMoreRansom.org"
            })
        
        if "T1490" in result.get("mitre_techniques", []):
            recommendations.append({
                "priority": 5,
                "action": "BACKUP_CHECK",
                "description": "Verify integrity of offline/immutable backups before recovery"
            })
        
        return sorted(recommendations, key=lambda x: x["priority"])

# Run advanced detection
detector = AdvancedRansomwareDetector()
detection_result = detector.analyze_attack(attack_simulation)

print("Advanced Ransomware Detection Results")
print("=" * 70)
print(f"Ransomware Detected: {detection_result['is_ransomware']}")
print(f"Confidence: {detection_result['confidence']:.1%}")
print(f"Risk Level: {detection_result['risk_level']}")
print(f"Detected Family: {detection_result['detected_family'] or 'Unknown'}")
print(f"Decryptor Available: {detection_result['decryptor_available']}")

print(f"\nIndicators Found: {len(detection_result['indicators'])}")
for ind in detection_result['indicators']:
    print(f"  [{ind['severity']}] {ind['type']}")

print(f"\nMITRE ATT&CK Techniques:")
for tech in detection_result['mitre_techniques']:
    info = MITRE_TECHNIQUES.get(tech, {})
    print(f"  {tech}: {info.get('name', 'Unknown')} ({info.get('tactic', '')})")

print(f"\nRecommendations:")
for rec in detection_result['recommendations']:
    print(f"  {rec['priority']}. [{rec['action']}] {rec['description']}")

## 4. Ransom Note Analysis & Family Classification

Analyze ransom notes to extract IOCs and classify ransomware families.

In [ ]:
# Sample ransom notes from different families
RANSOM_NOTES = {
    "lockbit": """
>>>>> Your data is stolen and encrypted <<<<<
Your personal DECRYPTION ID: 8A7F2C3D-9E1B-4F5A-BC6D-7E8F9A0B1C2D

The data will be published on TOR website if you do not pay the ransom.
You can buy decryption key from us, and we will delete stolen data.

Links for Tor Browser:
http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion
http://lockbit7z2jwcskxpbokpemdxmltipntwlkmidcll2qirbu7ykg46eyd.onion

>>>> DO NOT MODIFY ENCRYPTED FILES <<<<
>>>> DO NOT USE THIRD PARTY SOFTWARE TO RESTORE YOUR DATA <<<<

Contact email: support@lockbit.onion
Bitcoin wallet: bc1qxy2kgdygjrsqtzq2n0yrf2493p83kkfjhx0wlh

The price doubles after 72 hours.
""",
    
    "blackcat": """
>> Introduction
Important files on your system have been ENCRYPTED and now have the ".ALPHV" extension.

In order to recover your files you need to follow instructions below.

>> Sensitive Data
Sensitive data on your system was DOWNLOADED.
If you DON'T want sensitive data to be PUBLISHED you have to act quickly.

Data includes:
- Employees personal data, CVs, DL, SSN
- Complete network map
- Financial statements and reports
- Private and confidential agreements

>> What to do?
1. Download and install Tor Browser: https://www.torproject.org/
2. Open link in Tor Browser: alphvmmm27o3abo3r2mlmjrpdmzle3rykajqc5xsj7e7xqseahlawd.onion
3. Enter your KEY: X8F2A9B3C7D1E4F6A2B8C9D0E1F3A5B7

Monero (XMR) wallet: 
48Pu4y5gqFT8NHnM1fVPRZgNkFz8tGMJpzL6ZdSr3PeN...

DO NOT try to recover files yourself. DO NOT rename encrypted files.
""",
    
    "royal": """
If you're reading this, it means your company network was compromised.

All your important files have been encrypted with military grade encryption.
Any attempts to restore files using third party software will damage data.

What we took:
- Financial documents
- Customer databases  
- Employee information
- Intellectual property
- Email correspondence

You have 72 hours to contact us before we start publishing data.

Contact via Tor: royal2xthig3ou5hd7zs6hu25tejsrug65qylu2uoz3z2a3azxrrzad.onion
Email: royal@protonmail.com

Bitcoin: 1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
Ransom: $2,000,000 USD in Bitcoin

Time is money. Act fast.
""",
    
    "akira": """
Hi friends,

Whatever who you are and what your title is if you're reading this it means 
the internal infrastructure of your company is fully or partially dead.

All your files have been encrypted by Akira ransomware.

To get your data back you need to contact us and pay the ransom.

As a bonus we have downloaded a lot of interesting data from your network.

Data exfiltrated:
- Source code repositories
- Financial statements 2021-2024
- Customer PII database (2.3M records)
- Employee payroll data
- Strategic planning documents

Payment instructions:
1. Install Tor Browser
2. Visit: akiralkzxzq2dsrzsrvbr2xgbbu2wgsmxryd4csgfameg52n7efvr2id.onion
3. Enter your company ID: AK-2024-8834

Monero preferred, Bitcoin accepted.
Decryptor + data deletion = negotiable

Akira Team
""",

    "play": """
PLAY RANSOMWARE

Your network has been penetrated.

All files on each host in the network have been encrypted with a strong algorithm.

Backups were either encrypted or deleted.
Shadow copies also removed.

We have downloaded company data prior to encryption.

DO NOT:
- Rename encrypted files
- Modify encrypted files
- Use third party software to restore data
- Contact FBI, police, recovery companies

If you do - negotiations will end and data will be published immediately.

TO RESTORE DATA:
Email: play@onionmail.org
Tox ID: 8F2D4C1A...

Include your company name in subject.

After payment - you receive decryptor + proof of deletion.

Regards,
PLAY Team
"""
}

class AdvancedRansomNoteAnalyzer:
    """Advanced ransom note analysis with IOC extraction and family classification."""
    
    # IOC extraction patterns
    BTC_PATTERN = r'\b(bc1|[13])[a-zA-HJ-NP-Z0-9]{25,62}\b'
    MONERO_PATTERN = r'\b4[0-9AB][1-9A-HJ-NP-Za-km-z]{93}\b'
    ONION_PATTERN = r'\b[a-z2-7]{16,56}\.onion\b'
    EMAIL_PATTERN = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    TOX_PATTERN = r'\b[A-F0-9]{76}\b'
    SESSION_PATTERN = r'\b05[a-f0-9]{64}\b'
    UUID_PATTERN = r'\b[A-F0-9]{8}-[A-F0-9]{4}-[A-F0-9]{4}-[A-F0-9]{4}-[A-F0-9]{12}\b'
    
    # Family classification keywords
    FAMILY_SIGNATURES = {
        RansomwareFamily.LOCKBIT: {
            "keywords": ["lockbit", "lockbit 3.0", "lockbit 2.0", "lockbitapt", "restore-my-files"],
            "confidence_boost": ["lockbit7z", "decryption id"],
        },
        RansomwareFamily.BLACKCAT: {
            "keywords": ["alphv", "blackcat", ".alphv extension"],
            "confidence_boost": ["sensitive data", "tor browser"],
        },
        RansomwareFamily.CONTI: {
            "keywords": ["conti", "conti_readme", "contipay"],
            "confidence_boost": ["network compromised"],
        },
        RansomwareFamily.ROYAL: {
            "keywords": ["royal", "royal ransomware", "royal2xthig"],
            "confidence_boost": ["military grade encryption"],
        },
        RansomwareFamily.PLAY: {
            "keywords": ["play ransomware", "play team", "play@"],
            "confidence_boost": ["penetrated", "tox id"],
        },
        RansomwareFamily.AKIRA: {
            "keywords": ["akira", "akira ransomware", "akiralkzxzq"],
            "confidence_boost": ["hi friends", "dead"],
        },
    }
    
    def analyze(self, note: str) -> Dict:
        """Comprehensive ransom note analysis."""
        result = {
            "iocs": self._extract_iocs(note),
            "family": self._classify_family(note),
            "threat_intel": self._extract_threat_intel(note),
            "urgency_indicators": self._analyze_urgency(note),
            "data_exfil_claims": self._extract_exfil_claims(note),
        }
        return result
    
    def _extract_iocs(self, note: str) -> Dict:
        """Extract all IOCs from ransom note."""
        return {
            "bitcoin_addresses": list(set(re.findall(self.BTC_PATTERN, note))),
            "monero_addresses": list(set(re.findall(self.MONERO_PATTERN, note))),
            "onion_urls": list(set(re.findall(self.ONION_PATTERN, note, re.IGNORECASE))),
            "emails": list(set(re.findall(self.EMAIL_PATTERN, note))),
            "tox_ids": list(set(re.findall(self.TOX_PATTERN, note))),
            "victim_ids": list(set(re.findall(self.UUID_PATTERN, note, re.IGNORECASE))),
        }
    
    def _classify_family(self, note: str) -> Dict:
        """Classify ransomware family from note content."""
        note_lower = note.lower()
        
        best_match = None
        best_score = 0
        
        for family, sigs in self.FAMILY_SIGNATURES.items():
            score = 0
            matched_keywords = []
            
            for kw in sigs["keywords"]:
                if kw in note_lower:
                    score += 10
                    matched_keywords.append(kw)
            
            for boost in sigs.get("confidence_boost", []):
                if boost in note_lower:
                    score += 5
                    matched_keywords.append(boost)
            
            if score > best_score:
                best_score = score
                best_match = {
                    "family": family.value,
                    "confidence": min(score / 30, 1.0),
                    "matched_keywords": matched_keywords
                }
        
        if not best_match or best_score < 10:
            return {"family": "unknown", "confidence": 0.0, "matched_keywords": []}
        
        return best_match
    
    def _extract_threat_intel(self, note: str) -> Dict:
        """Extract threat intelligence from note."""
        # Payment demand patterns
        ransom_patterns = [
            r'\$[\d,]+(?:\s*(?:USD|usd))?',
            r'[\d.]+\s*(?:BTC|btc|Bitcoin)',
            r'[\d.]+\s*(?:XMR|xmr|Monero)',
        ]
        
        demands = []
        for pattern in ransom_patterns:
            matches = re.findall(pattern, note, re.IGNORECASE)
            demands.extend(matches)
        
        # Deadline patterns
        deadline_patterns = [
            r'(\d+)\s*hours?',
            r'(\d+)\s*days?',
        ]
        
        deadlines = []
        for pattern in deadline_patterns:
            matches = re.findall(pattern, note, re.IGNORECASE)
            deadlines.extend(matches)
        
        return {
            "ransom_demands": demands,
            "deadlines": deadlines,
            "double_extortion": any(
                phrase in note.lower() 
                for phrase in ["publish", "leak", "stolen", "downloaded", "exfiltrat"]
            )
        }
    
    def _analyze_urgency(self, note: str) -> Dict:
        """Analyze urgency tactics in the note."""
        urgency_phrases = {
            "time_pressure": ["hours", "days", "deadline", "time is running", "act fast", "quickly"],
            "price_escalation": ["price doubles", "price increases", "will increase"],
            "data_threats": ["publish", "leak", "expose", "sell", "auction"],
            "destruction_threats": ["delete", "destroy", "permanent", "unrecoverable"],
        }
        
        detected = {}
        note_lower = note.lower()
        
        for category, phrases in urgency_phrases.items():
            matches = [p for p in phrases if p in note_lower]
            if matches:
                detected[category] = matches
        
        return detected
    
    def _extract_exfil_claims(self, note: str) -> List[str]:
        """Extract claimed exfiltrated data types."""
        data_keywords = [
            "financial", "customer", "employee", "database", "source code",
            "contracts", "agreements", "payroll", "personal data", "pii",
            "ssn", "credit card", "intellectual property", "email"
        ]
        
        note_lower = note.lower()
        return [kw for kw in data_keywords if kw in note_lower]

# Analyze all sample notes
analyzer = AdvancedRansomNoteAnalyzer()

print("Ransom Note Analysis Results")
print("=" * 80)

for family_name, note in RANSOM_NOTES.items():
    analysis = analyzer.analyze(note)
    
    print(f"\n{'='*80}")
    print(f"FAMILY: {family_name.upper()}")
    print(f"{'='*80}")
    
    print(f"\nClassification: {analysis['family']['family']} (confidence: {analysis['family']['confidence']:.0%})")
    
    iocs = analysis['iocs']
    print(f"\nIOCs Extracted:")
    print(f"  Bitcoin addresses: {len(iocs['bitcoin_addresses'])}")
    print(f"  Onion URLs: {len(iocs['onion_urls'])}")
    print(f"  Emails: {iocs['emails']}")
    
    threat = analysis['threat_intel']
    print(f"\nThreat Intelligence:")
    print(f"  Ransom demands: {threat['ransom_demands']}")
    print(f"  Double extortion: {threat['double_extortion']}")
    
    if analysis['data_exfil_claims']:
        print(f"  Claimed data: {', '.join(analysis['data_exfil_claims'][:5])}")
    
    print(f"\nUrgency tactics: {list(analysis['urgency_indicators'].keys())}")

## 5. Incident Response Playbook Generator

Generate comprehensive response playbooks based on detection results.

In [ ]:
class IncidentResponsePlaybook:
    """Generate comprehensive ransomware incident response playbooks."""
    
    # Decryptor database (simplified - real would query NoMoreRansom.org)
    DECRYPTOR_DATABASE = {
        "conti": {"available": True, "source": "Leaked by affiliate", "url": "nomoreransom.org"},
        "akira": {"available": True, "source": "Avast", "url": "avast.com/ransomware-decryption-tools"},
        "lockbit": {"available": False, "notes": "No public decryptor available"},
        "blackcat": {"available": False, "notes": "FBI obtained some keys - check with law enforcement"},
        "hive": {"available": True, "source": "FBI operation", "url": "nomoreransom.org"},
    }
    
    def generate_playbook(self, detection: Dict, note_analysis: Dict = None) -> Dict:
        """Generate complete IR playbook."""
        family = detection.get("detected_family", "unknown")
        
        playbook = {
            "incident_id": f"IR-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
            "severity": detection.get("risk_level", "UNKNOWN"),
            "ransomware_family": family,
            "phases": []
        }
        
        # Phase 1: Immediate Containment (0-30 minutes)
        playbook["phases"].append({
            "phase": "CONTAINMENT",
            "timeframe": "0-30 minutes",
            "priority": "P0",
            "actions": [
                {"action": "Network isolation", "command": "Disable switch ports / Enable firewall block rules", "automated": True},
                {"action": "Disable compromised accounts", "command": "Disable-ADAccount -Identity <user>", "automated": True},
                {"action": "Block C2 domains/IPs", "command": "Add to firewall blocklist", "automated": True},
                {"action": "Alert IR team", "command": "PagerDuty/Slack notification", "automated": True},
                {"action": "Preserve volatile evidence", "command": "Memory dump with WinPMEM/DumpIt", "automated": False},
            ]
        })
        
        # Phase 2: Assessment (30 min - 2 hours)
        playbook["phases"].append({
            "phase": "ASSESSMENT",
            "timeframe": "30 min - 2 hours",
            "priority": "P1",
            "actions": [
                {"action": "Determine blast radius", "description": "Identify all affected hosts and shares"},
                {"action": "Identify encryption scope", "description": "Count encrypted files, affected departments"},
                {"action": "Analyze ransom note", "description": "Extract IOCs, determine if double extortion"},
                {"action": "Check backup integrity", "description": "Verify offline/immutable backups exist"},
                {"action": "Initial MITRE mapping", "description": f"Techniques detected: {detection.get('mitre_techniques', [])}"},
            ]
        })
        
        # Phase 3: Recovery Decision
        recovery_options = self._assess_recovery_options(family, detection)
        playbook["phases"].append({
            "phase": "RECOVERY_PLANNING",
            "timeframe": "2-4 hours",
            "priority": "P1",
            "recovery_options": recovery_options,
            "recommendation": recovery_options[0] if recovery_options else "Consult IR provider"
        })
        
        # Phase 4: Eradication
        playbook["phases"].append({
            "phase": "ERADICATION",
            "timeframe": "4-24 hours",
            "priority": "P2",
            "actions": [
                {"action": "Remove ransomware artifacts", "paths": ["C:\\ProgramData", "HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run"]},
                {"action": "Reset compromised credentials", "scope": "All privileged accounts + affected users"},
                {"action": "Patch exploitation vectors", "description": "Apply security updates to exploited systems"},
                {"action": "Re-image severely compromised hosts", "description": "Full rebuild for systems with persistence"},
            ]
        })
        
        # Phase 5: Recovery
        playbook["phases"].append({
            "phase": "RECOVERY",
            "timeframe": "1-7 days",
            "priority": "P2",
            "actions": [
                {"action": "Restore from clean backups", "verify": "Check backup date and integrity"},
                {"action": "Apply decryptor if available", "tool": self._get_decryptor_info(family)},
                {"action": "Validate data integrity", "method": "Compare file hashes with known good"},
                {"action": "Gradual network restoration", "sequence": "Critical systems first"},
            ]
        })
        
        # Phase 6: Post-Incident
        playbook["phases"].append({
            "phase": "POST_INCIDENT",
            "timeframe": "1-4 weeks",
            "priority": "P3",
            "actions": [
                {"action": "Root cause analysis", "deliverable": "Detailed incident report"},
                {"action": "Update detection rules", "deliverable": "New Sigma/YARA rules"},
                {"action": "Security improvements", "deliverable": "Remediation project plan"},
                {"action": "Tabletop exercise", "deliverable": "Lessons learned documentation"},
            ]
        })
        
        return playbook
    
    def _assess_recovery_options(self, family: str, detection: Dict) -> List[Dict]:
        """Assess recovery options based on situation."""
        options = []
        
        decryptor = self.DECRYPTOR_DATABASE.get(family, {})
        
        if decryptor.get("available"):
            options.append({
                "option": "USE_DECRYPTOR",
                "priority": 1,
                "description": f"Public decryptor available from {decryptor.get('source')}",
                "url": decryptor.get("url"),
                "cost": "Free"
            })
        
        options.append({
            "option": "RESTORE_BACKUP",
            "priority": 2,
            "description": "Restore from clean offline/immutable backups",
            "requirements": ["Verified clean backups", "Accept data loss to backup point"],
            "cost": "Time + potential data loss"
        })
        
        options.append({
            "option": "REBUILD",
            "priority": 3,
            "description": "Full rebuild of affected systems",
            "requirements": ["System images", "Application reinstall"],
            "cost": "High (time + resources)"
        })
        
        # Only include negotiation as last resort (not recommended)
        options.append({
            "option": "NEGOTIATION",
            "priority": 99,
            "description": "Last resort - engage with threat actor",
            "warnings": ["No guarantee of decryption", "May fund future attacks", "Check legal implications"],
            "cost": "Ransom amount"
        })
        
        return options
    
    def _get_decryptor_info(self, family: str) -> Dict:
        """Get decryptor information for family."""
        return self.DECRYPTOR_DATABASE.get(family, {"available": False, "notes": "No decryptor information available"})

# Generate playbook for our detection
playbook_gen = IncidentResponsePlaybook()
playbook = playbook_gen.generate_playbook(detection_result)

print("Incident Response Playbook")
print("=" * 80)
print(f"Incident ID: {playbook['incident_id']}")
print(f"Severity: {playbook['severity']}")
print(f"Family: {playbook['ransomware_family']}")

for phase in playbook["phases"]:
    print(f"\n{'='*60}")
    print(f"[{phase['priority']}] {phase['phase']} ({phase['timeframe']})")
    print(f"{'='*60}")
    
    if "actions" in phase:
        for action in phase["actions"][:4]:
            if isinstance(action, dict):
                print(f"  - {action.get('action', action)}")
    
    if "recovery_options" in phase:
        print("  Recovery Options:")
        for opt in phase["recovery_options"][:3]:
            print(f"    {opt['priority']}. {opt['option']}: {opt['description'][:50]}...")
        print(f"  RECOMMENDED: {phase['recommendation']['option']}")